In [1]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
# from collections import defaultdict
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
import string
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

## Loading data

In [2]:
#dataframe = pd.read_csv('all_essaysets.csv', encoding = 'latin-1')
dataframe = pd.read_csv('training.tsv', encoding = 'latin-1', sep='\t')
dataframe.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
count,12976.000000,12976.000000,12976.000000,12976.000000,128.000000,12976.000000,1800.000000,1800.000000,1800.000000,2292.000000,...,2292.000000,2292.000000,723.000000,723.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,10295.395808,4.179485,4.127158,4.137408,37.828125,6.800247,3.333889,3.330556,3.333889,2.444154,...,2.635689,2.710297,3.777317,3.589212,3.945312,3.890625,4.078125,3.992188,3.843750,3.617188
std,6309.074105,2.136913,4.212544,4.264330,5.240829,8.970705,0.729103,0.726807,0.729103,1.211730,...,1.142566,1.045795,0.689401,0.693256,0.643668,0.630390,0.622535,0.509687,0.538845,0.603417
min,1.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
25%,4438.750000,2.000000,2.000000,2.000000,36.000000,2.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000
50%,10044.500000,4.000000,3.000000,3.000000,40.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,15681.250000,6.000000,4.000000,4.000000,40.000000,8.000000,4.000000,4.000000,4.000000,3.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,21633.000000,8.000000,30.000000,30.000000,50.000000,60.000000,4.000000,4.000000,4.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,5.000000,5.000000


In [3]:
dataframe.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Methods

In [4]:
# selecting which set to be used 1-8
# in order to combine them all assign set number to 9
def select_set(dataframe,setNumber):
    if setNumber == 9:
        dataframe2 = dataframe[dataframe.essay_set ==1]
        texts = dataframe2['essay']
        scores = dataframe2['domain1_score']
        scores = scores.apply(lambda x: (x*3)/scores.max())
        for i in range(1,9):
            dataframe2 = dataframe[dataframe.essay_set == i]
            texts = texts.append(dataframe2['essay'])
            s = dataframe2['domain1_score']
            s = s.apply(lambda x: (x*3)/s.max())
            scores = scores.append(s)
    else:
        dataframe2 = dataframe[dataframe.essay_set ==setNumber]
        texts = dataframe2['essay']
        scores = dataframe2['domain1_score']
        scores = scores.apply(lambda x: (x*3)/scores.max())
    return texts, scores

In [5]:
# get histogram plot of scores and average score
def get_hist_avg(scores,bin_count):
    print(sum(scores)/len(scores))
    scores.hist(bins=bin_count)

In [6]:
#average word length for a text
def avg_word_len(text):
    clean_essay = re.sub(r'\W', ' ', text)
    words = nltk.word_tokenize(clean_essay)
    total = 0
    for word in words:
        total = total + len(word)
    average = total / len(words)
    
    return average

# word count in a given text
def word_count(text):
    clean_essay = re.sub(r'\W', ' ', text)
    return len(nltk.word_tokenize(clean_essay))

# char count in a given text
def char_count(text):
    return len(re.sub(r'\s', '', str(text).lower()))

# sentence count in a given text
def sent_count(text):
    return len(nltk.sent_tokenize(text))

#tokenization of texts to sentences
def sent_tokenize(text):
    stripped_essay = text.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
            tokens = nltk.word_tokenize(clean_sentence)
            tokenized_sentences.append(tokens)
    return tokenized_sentences


# lemma, noun, adjective, verb, adverb count for a given text

def count_lemmas(text):
    
    noun_count = 0
    adj_count = 0
    verb_count = 0
    adv_count = 0   
    lemmas = []
    lemmatizer = WordNetLemmatizer()
    tokenized_sentences = sent_tokenize(text)
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence) 
        
        for token_tuple in tagged_tokens:
            pos_tag = token_tuple[1]
            
            if pos_tag.startswith('N'): 
                noun_count += 1
                pos = wordnet.NOUN
                lemmas.append(lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('J'):
                adj_count += 1
                pos = wordnet.ADJ
                lemmas.append(lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('V'):
                verb_count += 1
                pos = wordnet.VERB
                lemmas.append(lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('R'):
                adv_count += 1
                pos = wordnet.ADV
                lemmas.append(lemmatizer.lemmatize(token_tuple[0], pos))
            else:
                pos = wordnet.NOUN
                lemmas.append(lemmatizer.lemmatize(token_tuple[0], pos))
    
    lemma_count = len(set(lemmas))
    
    return noun_count, adj_count, verb_count, adv_count, lemma_count

In [7]:
def token_word(text):
    text = "".join([ch.lower() for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return tokens

In [8]:
def misspell_count(text):
    spell = SpellChecker()
    # find those words that may be misspelled
    misspelled = spell.unknown(token_word(text))
    #print(misspelled)
    return len(misspelled)

In [9]:
def create_features(texts):
    data = pd.DataFrame(columns=('Average_Word_Length','Sentence_Count','Word_Count',
                                'Character_Count', 'Noun_Count','Adjective_Count',
                                'Verb_Count', 'Adverb_Count', 'Lemma_Count' , 'Misspell_Count'
                                 ))

    data['Average_Word_Length'] = texts.apply(avg_word_len)
    data['Sentence_Count'] = texts.apply(sent_count)
    data['Word_Count'] = texts.apply(word_count)
    data['Character_Count'] = texts.apply(char_count)
    temp=texts.apply(count_lemmas)
    noun_count,adj_count,verb_count,adverb_count,lemma_count = zip(*temp)
    data['Noun_Count'] = noun_count
    data['Adjective_Count'] = adj_count
    data['Verb_Count'] = verb_count
    data['Adverb_Count'] = adverb_count
    data['Lemma_Count'] = lemma_count
    data['Misspell_Count'] = texts.apply(misspell_count)
    return data

In [10]:
def data_prepare(texts,scores):
    #create features from the texts and clean non graded essays
    data = create_features(texts)
    data.describe()
    t1=np.where(np.asanyarray(np.isnan(scores)))
    scores=scores.drop(scores.index[t1])
    data=data.drop(scores.index[t1])
    
    #scaler = MinMaxScaler()
    #data = scaler.fit_transform(data)

    #train test split
    X_train, X_test, y_train, y_test = train_test_split(data, scores, test_size = 0.3)

    #checking is there any nan cells
    print(np.any(np.isnan(scores)))
    print(np.all(np.isfinite(scores)))
    return X_train, X_test, y_train, y_test, data

In [11]:
def lin_regression(X_train,y_train,X_test,y_test):
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)

    # The mean squared error
    mse=mean_squared_error(y_test, y_pred)
    mse_per= 100*mse/3
    print("Mean squared error: {}".format(mse))
    print("Mean squared error in percentage: {}".format(mse_per))
    #explained variance score
    print('Variance score: {}'.format(regr.score(X_test, y_test)))

## Dataset selection

In [12]:
# 1-8
# 9:all sets combined
texts, scores = select_set(dataframe,1)
# get_hist_avg(scores,5)
X_train, X_test, y_train, y_test, data = data_prepare(texts,scores)

False
True


## Regression Analysis

In [19]:
print('Testing for Linear Regression \n')
# lin_regression(X_train,y_train,X_test,y_test)
regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

# The mean squared error
mse=mean_squared_error(y_test, y_pred)
mse_per= 100*mse/3
print("Mean squared error: {}".format(mse))
print("Mean squared error in percentage: {}".format(mse_per))
#explained variance score
print('Variance score: {}'.format(regr.score(X_test, y_test)))

Testing for Linear Regression 

Mean squared error: 0.03955152150487646
Mean squared error in percentage: 1.3183840501625486
Variance score: 0.7069499262501784


In [20]:
################ BUILD

In [21]:
def prediction(text):
    text = pd.Series(text)
    text = create_features(text)
    result = regr.predict(text)[0]
    print(result)

In [23]:
# # New test case
# case1 = "Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy, sea floor spreading or even about the @DATE1's you'll be surprise at how much he/she knows. Believe it or not the computer is much interesting then in class all day reading out of books. If your child is home on your computer or at a local library, it's better than being out with friends being fresh, or being perpressured to doing something they know isnt right. You might not know where your child is, @CAPS2 forbidde in a hospital bed because of a drive-by. Rather than your child on the computer learning, chatting or just playing games, safe and sound in your home or community place. Now I hope you have reached a point to understand and agree with me, because computers can have great effects on you or child because it gives us time to chat with friends/new people, helps us learn about the globe and believe or not keeps us out of troble. Thank you for listening."
# case2 = """Dear Local Newspaper, @CAPS1 I have found that many experts say that computers do not benifit our society. In some cases this is true but in most cases studdies show that computers can help people. While nothing beats talking in person computers can get really close such examples are webcams or e-mail. @PERCENT1 of students who get good grades refer to reliable websites for reasearch or to help find good books. Also online catalouges or advertisments help the economy of stores worldwide. @CAPS2 people were not allowed to use computers most of the modern would not exist. @PERSON1 said that the best form of modern communication is the computer because of the ability to write, talk, or write back for much cheaper! Almost every single event i go to is planed on a computer by communication such as e-mail "@CAPS2 a student ever needs homework because lam out sick or needs help studying for a test then contact their teacher through the best form of communication for them always e-mail. Even the post office uses computers to get letters and boxes to people. The president of the post office, @PERSON2 said "@CAPS3 would be imposible to get mail to our coustmers @CAPS2 @CAPS3 were not for computers telling us where a zip code is or how heavy a box is." @CAPS4 that tell people what is happening around the world would not exist @CAPS2 @CAPS3 were not for the moder communication abilities that computer provid us. Because information can be commucated so quick. so can reasearch. When the country of @LOCATION2 took a pole @PERCENT2 of people used computer for any type of reasearch, of those @PERCENT3 were students currently in school and @PERCENT4 of them have good grades. When the same survey was taken in the @LOCATION1 @PERCENT5 of people used computers fore reasons and @PERCENT2 were students who had good grade @CAPS2 @CAPS3 were not posible for me to access documents in the @CAPS5 @CAPS6 online I probably would not have gotten an A+ on my @CAPS7 assignment! Could you amagine @CAPS2 suddenly your Newspaper reporters couldn't use the internet to work on their reports? The articles would probably be @NUM1 after the events occur. Most buissness, including the Newspaper, use the internet to advertise, shop, or read. The association of @ORGANIZATION1 reported that in @PERCENT1 of @ORGANIZATION1 used a website and of them @PERCENT5 were in good positions. The president of @CAPS8 @NUM2 imports said that they use an online catalouge because @CAPS3 is cheaper, but they can also promote that @CAPS3 is to save trees, or for the castomer's convinence. Small @ORGANIZATION1 can make websites to promote them selves and explain their star to potential coustomers. @PERSON3, the owner of @ORGANIZATION2's said that the internet saved her resturant. @CAPS2 @CAPS3 wer not for the internet @NUM3 more people would be jobless in @LOCATION3. In conclusion computer help everyday people and without them most convinences would not exist. They help communicate around the world. Computers help people reaserch subjects for school reports, and they make the current economy get better everyday. In moderation computers are the most useful tool out there."""
# case3 = """The affects of the cyclist is if it does not change. it cut hurt a lot of people feeling because they @CAPS1 don’t care about the cyclist. I’m one of them people who does not care about it cause it does not affect me or anyone I know. It is a big deal to write people, some of them @CAPS1 blow stuff up. They talking on tiv and on the radio making all this stuff they say is made up. I don’t believe to I see it. That @CAPS1 not for black people because it has not did anything to us. We really don’t care about the affects of the cyclist."""
# case4 = """Understanding, realitic, mode-all are reason’s how setting affects the setting. If a book has a good setting then the book might be pretty good. When you have a setting it’s more understable of what they are trying today. And if you have a good setting it makes it more realitic So when you read it you wont be thrown off, the mode of the story sets it all off, because usually books have there ups and downs. That’s why they chase the mode.  """
# case5 = """ Bell rings.  Shuffle, shuffle. @CAPS1. Snap. EEEE. Crack. Slam. Click, stomp, @CAPS1. Tap tap tap. SLAM. Creak. Shoof, shoof.  Sigh. Seventh class of the day. Here we go. "@CAPS2! Tu va ou pas? On a +¬tude cette class-l+á. Tu peux aller au bibliotheque si tu veux...." @CAPS3 all blinked at me, @PERSON1, @NUM1le and @ORGANIZATION1, chocolate-haired and mocha skinned, impatiently awaiting my answer. The truth was, I knew @CAPS3 didn't really care if I came or not. It made no difference to them if I trailed a few feet behind like some pathetic puppy. I was silent but adorable, loved only because I was an @CAPS4. Because I spoke fidgety @CAPS5. Because I was the exchange student, because my translator and colorful clothes were so shocking for ten seconds, and were then forgotten about.  I was a flock of seagulls haircut. So why are you here? I thought. Why did you go on exchange at all? You are the complete opposite of everyone here. No one wants you. Just go home.  But my ego had a ready answer. You begged for this remember? For months and months, it was all you wanted, all you thought about, all you dreamt about. So I went with the girls. As expected, @CAPS3 walked down the three-person wide staircase side-by-side, and I shuffled awkwardly behind them. Finally arriving at @NUM2scalier, we sat at a table, the three girls talking. I glazed my eyes over, attempting to look lost in thought, as if I didn't care I wasn't included. Selfish thoughts buzzed in my head; if @CAPS3 weren't talking to me, why should I make the effort to talk to them?   I really had no idea how @CAPS3 felt about me. How does someone feel about their shadow? @CAPS3 notice it, sure, but it never offers up insight, it never makes you laugh. It's all in the confidence, said my mother's voice, all how you carry yourself. But I knew it wasn't that simple. I was just too alien. These girls would never understand me, as I would never understand them. In frustration, I started to flick peas across the room with my spoon. Pat, flick, sproing.  This caught the interest of @PERSON1, as @NUM1le and @ORGANIZATION1 were discussing something very emotional. Tears began to pour out of @ORGANIZATION1's eyes. Sniffling, she and @NUM1le went to the bathroom, leaving me all alone with @PERSON1. Only @PERSON2 could have felt my felt my same emotion as he stared up at @CAPS6. Silently, I continued shooting peas. @PERSON1 just stared at them as @CAPS3 darted around the room. Suddenly, with a horrible miscalculation, a pea hit a boy in the face. And then, he turned around and swore. And then, @PERSON1 and I looked at each other from across the table.  And then, we laughed.  We laughed so hard I cried. So hard that huge, alien tears flooded from my eyes. People around us were laughing too, even though @CAPS3 had no idea what was so funny. I didn't even know what was so funny. But it didn't matter, because we were dripping tears and snot, reaching for each other, reenacting the pea hitting the boy's face. It was as if we had been friends for years, and laughing happened all the time. It was saturated with all the angst and lonliness and despair I had felt the past four weeks. The connection we felt was instantaneous, like lightening, the kind of connection I felt with my best friends back home. I felt that huge swelling sensation in my chest, like a balloon was stuck inside. My stomach was aching and my cheeks were so sore I felt them seizing up. My heart felt whole even for that second. My soul was open. It was the best laugh of my life.  Sniffle sniffle. GASP. Laughter. GASP. Swipe of tears. Sniffle sniffle. Laughter. GASP. This is why. I thought. This is why you came.  Bell rings."""

# prediction(case1)
# prediction(case2)
# prediction(case3)
# prediction(case4)
# prediction(case5)

# # if result < 2:
# #     print("Bad")
# # elif result < 3:
# #     print("Average")
# # elif result < 4:
# #     print("Good")

2.0520967125290177
2.6428213588460525
1.3589641359255733
1.3067274047407462
3.1603094510224405


In [26]:
import pickle
with open("predictModel.sav", "wb") as file:
    pickle.dump(regr, file)

with open('predictModel.sav', 'rb') as file:
    MODEL = pickle.load(file)

In [31]:
pred_text = pd.Series(case1)
pred_text = create_features(pred_text)
MODEL.predict(pred_text)[0]

2.0520967125290177